Using mobilenet v2 for making a classifier for alpaca or not

In [123]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

Prepare dataset first from the directory

In [124]:
directory='dataset/'
IMAGE_SIZE=(160,160)
BATCH_SIZE=32
train_dataset=tf.keras.preprocessing.image_dataset_from_directory(directory=directory,shuffle=True,image_size=IMAGE_SIZE,seed=42,validation_split=0.2,subset='training',batch_size=BATCH_SIZE)
validation_dataset=tf.keras.preprocessing.image_dataset_from_directory(directory=directory,shuffle=True,image_size=IMAGE_SIZE,seed=42,validation_split=0.2,subset='validation',batch_size=BATCH_SIZE)

Found 327 files belonging to 2 classes.


Using 262 files for training.
Found 327 files belonging to 2 classes.
Using 65 files for validation.


define data augmentator and preprocessing

In [125]:
AUTOTUNE=tf.data.AUTOTUNE
train_dataset=train_dataset.prefetch(AUTOTUNE)

In [126]:
def data_augmentator():
    data_augment=tf.keras.Sequential()
    data_augment.add(tf.keras.layers.RandomFlip('horizontal'))
    data_augment.add(tf.keras.layers.RandomRotation(factor=0.2))
    return data_augment

In [127]:
data_augmentation=data_augmentator()
preprocess_input=tf.keras.applications.mobilenet_v2.preprocess_input

In [139]:
base_model=tf.keras.applications.MobileNetV2(input_shape=IMAGE_SIZE+(3,),include_top=False,weights='imagenet')


In [140]:
base_model.trainable=False
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_62 (InputLayer)       [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 80, 80, 32)           864       ['input_62[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 80, 80, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 80, 80, 32)           0         ['bn_Conv1[

In [141]:
def alpaca_model(image_shape,data_augmentation=data_augmentator()):
    input_shape=image_shape+(3,)
    
    inputs=tf.keras.Input(shape=input_shape)
    #Preprocessing
    X=data_augmentation(inputs)
    X=preprocess_input(X)
    print(X.shape)
    
    X=base_model(X,training=False)
    X=tf.keras.layers.GlobalAveragePooling2D()(X)
    print(X.shape)
    X=tf.keras.layers.Dropout(rate=0.2)(X)
    pred_layer=tf.keras.layers.Dense(1)
    X_out=pred_layer(X)
    print(X_out.shape)

    model=tf.keras.Model(inputs=inputs,outputs=X_out)
    return model

In [144]:
initial_epoch=5
model1=alpaca_model(IMAGE_SIZE)
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
               metrics=['accuracy'])
model1.summary()

(None, 160, 160, 3)
(None, 1280)
(None, 1)
Model: "model_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_64 (InputLayer)       [(None, 160, 160, 3)]     0         
                                                                 
 sequential_12 (Sequential)  (None, 160, 160, 3)       0         
                                                                 
 tf.math.truediv_32 (TFOpLa  (None, 160, 160, 3)       0         
 mbda)                                                           
                                                                 
 tf.math.subtract_32 (TFOpL  (None, 160, 160, 3)       0         
 ambda)                                                          
                                                                 
 mobilenetv2_1.00_160 (Func  (None, 5, 5, 1280)        2257984   
 tional)                                                         
               

In [145]:
history=model1.fit(train_dataset,epochs=initial_epoch,validation_data=validation_dataset)

Epoch 1/5


9/9 [==============================] - 6s 400ms/step - loss: 0.8501 - accuracy: 0.6069 - val_loss: 0.5537 - val_accuracy: 0.7077
Epoch 2/5
9/9 [==============================] - 3s 306ms/step - loss: 0.5113 - accuracy: 0.7863 - val_loss: 0.3088 - val_accuracy: 0.8769
Epoch 3/5
9/9 [==============================] - 3s 310ms/step - loss: 0.3320 - accuracy: 0.8206 - val_loss: 0.1107 - val_accuracy: 0.9538
Epoch 4/5
9/9 [==============================] - 3s 311ms/step - loss: 0.2715 - accuracy: 0.8740 - val_loss: 0.1156 - val_accuracy: 0.9385
Epoch 5/5
9/9 [==============================] - 3s 307ms/step - loss: 0.2188 - accuracy: 0.8931 - val_loss: 0.0861 - val_accuracy: 0.9385


In [146]:
base_model.trainable=True
fine_tune_at=126
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable=False
    


In [136]:
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_60 (InputLayer)       [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 80, 80, 32)           864       ['input_60[0][0]']            
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 80, 80, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 80, 80, 32)           0         ['bn_Conv1[

In [147]:
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
               metrics=['accuracy'])

In [165]:
epochs=5
total_epoch=initial_epoch+epochs
history_new=model1.fit(train_dataset,validation_data=validation_dataset,epochs=total_epoch,initial_epoch=history.epoch[-1])

Epoch 5/10


9/9 [==============================] - 4s 405ms/step - loss: 0.0299 - accuracy: 0.9885 - val_loss: 0.0438 - val_accuracy: 0.9846
Epoch 6/10
9/9 [==============================] - 4s 405ms/step - loss: 0.0399 - accuracy: 0.9809 - val_loss: 0.0584 - val_accuracy: 0.9846
Epoch 7/10
9/9 [==============================] - 5s 477ms/step - loss: 0.0361 - accuracy: 0.9809 - val_loss: 0.0283 - val_accuracy: 1.0000
Epoch 8/10
9/9 [==============================] - 4s 404ms/step - loss: 0.0196 - accuracy: 0.9924 - val_loss: 0.0261 - val_accuracy: 1.0000
Epoch 9/10
9/9 [==============================] - 4s 407ms/step - loss: 0.0113 - accuracy: 1.0000 - val_loss: 0.0259 - val_accuracy: 0.9846
Epoch 10/10
9/9 [==============================] - 4s 355ms/step - loss: 0.0173 - accuracy: 0.9962 - val_loss: 0.0296 - val_accuracy: 0.9692
